## Package Import

In [2]:
import itertools
import random
import subprocess
import os

## Procedure
To show that 3-CAP is NP-hard, we give a polytime reduction from 3-COL to 3-CAP. That is, given an instance of 3-COL (i.e. a graph G(V, E)), we will construct an instance G_CAP of 3-cap where G_cap is 3-allocable iff G is 3-colorable.</br>

## Contrustruction

Given a graph G, for each edge (x,y), we create a conflict by 
- viewing x, y as two modules Mx and My
- creating two requirements Rx,Ry s.t. conflict(Rx,Ry)
- add CAP-edges (Mx,Rx) and (My, Ry)

![Construction](./img/construction.png)

## Approach
Then we get a CAP-graph G_cap
Claim: G_CAP is 3-allocable iff G is 3-colorable

We can interprate the claim in two ways [relevant material](http://www3.govst.edu/wrudloff/CPSC438/CPSC438/CH01/Chapter1/Section.1.1.pdf) </br>

⇒ **`if G is 3-col, then G_cap is 3-alloc`** = `if G_cap is not 3-alloc then G is not 3-col` </br>

⇐ `if G is not 3-col, then G_cap is not 3-alloc` = **`if G_cap is 3-alloc then G is 3-col`**

## Experiments

### Thoery
Suppose G is 3-colorable, s.t. no neighbors have the same color. This 3-coloring defines a 3-alloc (in the obvious way): the color of a node (module) uniquely identifies a box on which that module can be installed. A conflict could exist if two reqs R1 and R2 that are in conflict appear on the same box. However, each Ri inherits the color of the unique module on which it is used.

### Emperical Confirmation with ASP

#### COL Generation

In [3]:
def graph_generate(node_num,edge_num,graph_seq=1):
    node_ids=[node_id for node_id in range(1,node_num+1) ]# prepare for itertools
    # generate all edges(undirected without self-connected)
    edges=list(itertools.combinations(node_ids, 2)) 
    graph_ls=[]
    for seq in range(graph_seq):
        graph=random.sample(edges, edge_num)
        graph_ls.append(graph)
#     print(graph_ls)
    return(graph_ls)

In [4]:
def graph_clingo_generate(node_num,edge_num,col_num,graphs_ls,graph_seq=1): 
    newpath = 'clingo_files'
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    for graph_id in range(graph_seq):
        graph=graphs_ls[graph_id] # randome selected graph (tuple)
        clingo=[] # add all elements to the clingo list
        for edge in graph:
            edge_temp="edge"+str(edge)+"."
            clingo.append(edge_temp)
        clingo.append("#const n = "+str(col_num)+".")
        clingo.append("{ color(X,1..n) } = 1 :- node(X).")
        clingo.append("node(X) :- edge(X,_).")
        clingo.append("node(X) :- edge(_,X).")
        clingo.append(":- edge(X,Y), color(X,C), color(Y,C).")
        clingo_script='\n'.join(clingo)
        
        with open(newpath+"/col_clingo"+".lp", "w") as text_file:
            text_file.write(clingo_script)
#     return(clingo)

#### CAP Generation

In [5]:
def cap_clingo_generate(node_num,edge_num,box_num,graphs_ls,graph_seq=1): 
    newpath = 'clingo_files'
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    for graph_id in range(graph_seq):
        graph=graphs_ls[graph_id] # randome selected graph (tuple)
        clingo=[] # add all elements to the clingo list
        for edge in graph:
            edge_temp="edge"+str(edge)+"."
            clingo.append(edge_temp)
        clingo.append("m(X) :- edge(X,_).")
        clingo.append("m(X) :- edge(_,X).")
        clingo.append("con(X,Y):-edge(X,Y).")
        clingo.append("req (X):- con(X,_).")
        clingo.append("req (X):- con(_,X).")
        clingo.append("uses(X,Y):- m(X),req(Y),X=Y.")
        clingo.append('box(1..'+str(box_num)+').')
        clingo.append("1 { on(B,M) : box(B) } 1  :- m(M).")
        clingo.append("inst(B,R) :- on(B,M), uses(M,R).")
        clingo.append(":- con(R1,R2), R1 != R2, inst(B,R1), inst(B,R2).")
        clingo_script='\n'.join(clingo)
        
        with open(newpath+"/cap_clingo"+".lp", "w") as text_file:
            text_file.write(clingo_script)
#     return(clingo)

#### COL_CAP Validation

In [6]:
def clingo_generate(node_num,edge_num,col_box_num,graph_seq=1):
    graphs_ls=graph_generate(node_num,edge_num,graph_seq) # all graphs
    graph_clingo_generate(node_num,edge_num,col_box_num,graphs_ls,graph_seq=1)
    cap_clingo_generate(node_num,edge_num,col_box_num,graphs_ls,graph_seq=1)

In [7]:
def cmd_result(command):
    stdout = subprocess.getoutput(command)
    result=stdout.split("\n")[-6]
    return(result)

In [8]:
def col_cap(node_num,edge_num,col_box_num):
    clingo_generate(node_num,edge_num,col_box_num)
    graph_command='bash -c "clingo clingo_files/col_clingo.lp"'
    cap_command = 'bash -c "clingo clingo_files/cap_clingo.lp"'
#     print(cmd_result(graph_command),cmd_result(cap_command))
    if cmd_result(graph_command)!=cmd_result(cap_command):
        print("ALERT: COL and CAP mismacth")

### Result

In [9]:
for i in range(100):
    col_cap(5,8,3)    #node_num, edge_num, col number or box number